## Latest summary of objectives

- Develop an efficient solver.
- Develop a grid generator that satisfies some measures of symmetry, randomness, uniformity, etc.
- Analyze the number of solutions for various grids.
- Create generators that produce grids with different distributions of solutions.

## 2025-12-03: Intro

I saw my kid playing <a href="https://youtube.com/watch?v=VAdmJ525lXY">Stitch.</a>, so naturally I wanted to write a solver for it. If I understand correctly, the game is based on <a href="https://en.wikipedia.org/wiki/Shikaku">Shikaku</a>:

> Shikaku is played on a rectangular grid. Some of the squares in the grid are numbered. The objective is to divide the grid into rectangular $\dots$ pieces such that each piece contains exactly one number, and that number represents the area of the rectangle.

I will try to consistently use terms as follows:
- **Grid** is used as described above and refers to the puzzle as presented to the player. It is composed of squares and contains numbers but not rectangles.
- **Square** refers to an individual cell in the grid.
- **Rectangle** refers to a solid rectangle formed by one or more squares.
- **Number** refers to one of the numbers in the grid. It resides in a particular square and in a solution is associated with a particular rectangle.

### Thoughts about a solver

Which of the following will be more efficient?

- choosing as the next step in the search a particular square containing a number, or a particular square regardless of whether it contains a number, in either case finding all possible rectangles containing that square
- if choosing only squares with numbers — starting from larger numbers, or smaller numbers, or prime numbers, or numbers with the most factors
- choosing a next step adjacent to or distant from the previous step
- starting from the inside or the outside
- breadth-first search or depth-first search

For example, would doing all four corners first be more efficient? Walking the perimeter? Starting from the largest numbers?

Also, what should I compute in advance?

- Which numbers' rectangle could overlap with a given square?
- Should I skip right to which rectangles could overlap with a given square?
- Is it practical to cache results for a given partial solution?
- Are there arrangements of numbers that can be efficiently reduced into their possible combinations of rectangles? (My gut says no, not efficiently.)

How large of a grid am I willing to consider? How high of numbers will I allow?

And I quickly realized that generating sample grids is not at all trivial.

### Thoughts about grid generation

I have the sense that attempting to generate a "random" grid is <a href="https://www.youtube.com/watch?v=mZBwsm6B280">almost a meaningless statement</a>.

Let's look at how many possible 2 $\times$ 2 grids there are. For any given collection of rectanges, each rectangle's number $n$ could have been be placed in any of the $n$ squares within the rectangle. This means the quantity of grids produced by any given collection of rectangles is equal to the product of the rectangles' areas, or numbers, less any duplication that may occur from different collections of rectangles producing the same grid. These are the possible combinations of rectangle areas and how many distinct ways they can occur:

- (4,) — 1 distinct partition
- (2, 2) — 2 distinct partitions, vertically or horizontally
- (2, 1, 1) — 4 distinct partitions, as the 2 can be vertical or horizontal and in the first or second row/column
- (1, 1, 1, 1) — 1 distinct partition

These suggest 21 total grids from these 8 distinct partitions, but only 19 of the grids are distinct. In the (2, 2) case, the numbers can be on a diagonal whether the rectangles are vertical or horizontal, so instead of $(2 \times 2) \times 2 = 8$ there are only 6 distinct grids from this case.

Still, that's 19 distinct grids from 8 distinct partitions, and most of these partitions have a significant number of rectangles of size 1. (My kid says size 1 is allowed, but I haven't seen this in the examples I've found online.) I imagine that grids-to-partitions ratio may grow as the size of the grid grows. And even if it doesn't, <a href="https://oeis.org/A333476">OEIS A333476</a> indicates that there are over 84 million distinct partitions for a $5 \times 5$ rectangle.

Getting back to grid generation — what would a "random $5 \times 5$ grid" mean? Would it mean generating all 84 million distinct partitions, and all their however-many distinct grids, and then randomly selecting one? That's not a practical approach. It's also probably misguided. To the extent that there is some real "population" of grids that I would try to make my solver efficient at solving, those grids would have been created by some algorithm, and I'd effectively be trying to figure out what that algorithm might be.

Perhaps as I go about testing my own algorithms for grid generation, I can evaluate them with statistics. It's not clear to me whether on average I should expect the locations of numbers to be uniformly distributed throughout the grid, but I would expect numbers to be equally likely between symmetrical positions. For example, there shouldn't be a bias toward the upper-left.

Also, I may be able to find enough examples of puzzles intended for humans to solve that I can establish a preference for the relative frequency of different numbers. On average there should probably be more 3s than 9s, but how much more? Should there be more 7s or 8s?

I'm also interested in how many solutions a grid has. One of my favorite puzzle games, <a href="https://en.wikipedia.org/wiki/Flow_Free">Flow Free</a>, seems to limit its puzzles to those with a unique solution. I wonder whether, for example, the numbers in a grid, or their positions, may be correlated with how many solutions the grid has. And then by tweaking the generation algorithm, I would be more likely to create grids with very few or very many solutions.

### Objectives

- Write a solver. Make it quantifiably better.
- Write a grid generator. Make it statistically defensible.
- Analyze the number of solutions for various grids. Tweak the generator as desired without losing its statistical defensibility.

## 2025-12-04: Bit masking and preprocessing

I've decided I will only consider rectangles of size 2 or greater, partly because this seems to be the convention for published puzzles. Size 1 rectangles are equivalent to holes in the grid (or free spaces). Handling non-rectangular grids or grids with holes feels like a straightforward enhancement of what I'll eventually build, but I'm more interested in fine-tuning the algorithm and analyzing the statistics, so I don't know that I'll get to it. But if I do, it's effectively the same as a rectangular puzzle with the number 1 everywhere within the bounding box but outside the grid.

Now a big part of making the eventual algorithm efficient will be checking whether two rectangles overlap, or whether a rectangle contains a particular square. I'll be using Python, but any use of lists or tuples or even NumPy arrays will be very slow. Bitwise operations in Python are very fast, so a more efficient approach is to use a bit mask. Any rectangle or collection of rectangles overlaps a collection of squares. We can represent whether it overlaps any given square with just one bit: 1 for yes, 0 for no. If all squares in the grid are always considered in the same order, those bits can be interpreted as a binary number. If there is any bit that is set to 1 in both of the binary numbers representing two objects, the objects both contain the square associated with that bit and therefore the two objects overlap. For example:

```
for candidate in candidates:
    if candidate & collection:
        continue
    do_stuff()
```

This would skip any candidate that overlaps with the existing collection.

I also think it will turn out to be efficient to compute in advance the bitmask for each possible rectangle for a given grid. For each number $n$ in the grid:

- Compute all pairs of factors $(x,y)$ such that $x \times y = n$. For $n = 6$ these pairs would be $(1,6)$, $(2,3)$, $(3,2)$, $(6,1)$.
- For each pair of factors, consider each rectangle of that shape that overlaps with the sqaure containing the number.
- Any rectangle that overlaps another number is not possible.
- Any rectangle that extends beyond the grid is not possible.
- Any remaining rectangle is possible.

## 2025-12-05: Initial thoughts on representing the grid

Here is the <a href="https://en.wikipedia.org/wiki/Shikaku#/media/File:Shikaku_start.png">sample puzzle</a> on the Wikipedia page at the time I'm writing this:
```
+--+--+--+--+--+--+--+--+--+--+--+
|  | 4|  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2| 4|  |  | 3|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |16|  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 6| 5| 2| 4|  |  | 4|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  | 2| 5|  |  | 2|  | 7|
+--+--+--+--+--+--+--+--+--+--+--+
|  |  | 2|  |  | 2| 4|  | 2|  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |16|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |12|  | 6|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2|  |  |  |  |  |  |  |  | 9|  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
```
It would be nice if the printed representation of the grid/puzzle could be read back in as that puzzle. I might add that at some point, but for now I'll parse it down to something simpler to work with.

In [1]:
sample_grid_repr = """
+--+--+--+--+--+--+--+--+--+--+--+
|  | 4|  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2| 4|  |  | 3|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |16|  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 6| 5| 2| 4|  |  | 4|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  | 2| 5|  |  | 2|  | 7|
+--+--+--+--+--+--+--+--+--+--+--+
|  |  | 2|  |  | 2| 4|  | 2|  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |16|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |12|  | 6|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2|  |  |  |  |  |  |  |  | 9|  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
"""

sample_grid_cleaned = '\n'.join(
    line[1:-1].replace('|', ',').replace(' ', '')
    for line in sample_grid_repr.strip().split('\n')[1::2])

print(sample_grid_cleaned)

,4,,,,,,,,,
2,4,,,3,,,,,,
,,,,,,,16,,,
6,5,2,4,,,4,,,,
,,,,2,5,,,2,,7
,,2,,,2,4,,2,,
,,,,16,,,,,,
,,,,,,,,,,
,,,,12,,6,,,,
2,,,,,,,,,9,
,,,,,,,,,,


To make it straightforward to generate the bit mask for each rectangle, I think I'll want to use NumPy arrays. For example, for a grid with `r` rows and `c` columns:

```
import numpy as np
bitmask = np.array([2 ** i for i in range(r * c)]).reshape((r, c))
```

I'll want to compute the possible rectangle sizes for a given number:

```
from sympy import divisors
factors = divisors(number)
shapes = list(zip(factors, factors[::-1]))
```

TO DO:

- iterate all possible positions/offsets of the rectangle of each shape for each number
- filter out any that extend beyond the grid or overlap any other number
- add each remaining rectangle to any appropriate maps

I'll want at least a couple of maps (dicts in Python):

- squares with numbers -> that square's number
- squares -> list of possible rectangles containing that square

## 2025-12-06: A couple of thoughts

A solvable puzzle will remain solvable if a number is moved to any other square in its rectangle. This means a solvable collection of rectangles can have a ridiculously large number of grid variants. How to choose among them? Randomly sample, test for number of solutions, compute statistics, ...?

In [2]:
import math

variants = math.prod(int(n) for n in sample_grid_cleaned.replace(',', ' ').split())
print(f"Sample grid has {variants:,} variants.")

Sample grid has 136,982,613,196,800 variants.


A grid generator where number 1 is not allowed can reach a dead end. I may want to track how many dead ends are reached before a solvable grid is returned.

## 2026-01-01 First solver

Picking this back up after <a href="https://adventofcode.com/2025">Advent of Code</a> and the holidays.

While thinking about a solver over the past few days, I thought in terms of additional for-loops and not NumPy arrays. So I wrote everything using for-loops and the standard library.

First is a helper function for identifying all factors for a given number. I could make this more efficient by precomputing all pairs of factors and accessing them from a list by index. Or at least using `functools.cache()`. I'll optimize later.

In [3]:
def factors(n):
    return [i for i in range(1, n + 1) if n % i == 0]

assert factors(12) == [1, 2, 3, 4, 6, 12]
assert factors(13) == [1, 13]

Next is computing the bitmask, relative to the grid, for all squares in a rectangle. Zero is returned if the rectangle doesn't stay within the grid. I moved this inside the class I decided to create, but I left this version here because I tested a handful of cases. I'm trying to show some of the thought process here...

In [4]:
def bitmask_rectangle(H, W, r, c, h, w):
    if r < 0 or c < 0 or r + h > H or c + w > W:
        return 0
    result = 0
    for r0 in range(r, r + h):
        for c0 in range(c, c + w):
            result += 2 ** (r0 * W + c0)
    return result

assert bitmask_rectangle(3, 4, -1, 1, 1, 2) == 0
assert bitmask_rectangle(3, 4, 1, -1, 1, 2) == 0
assert bitmask_rectangle(3, 4, 1, 1, 1, 2) == 96
assert bitmask_rectangle(3, 4, 1, 1, 3, 1) == 0
assert bitmask_rectangle(3, 4, 1, 1, 1, 4) == 0
assert bitmask_rectangle(3, 4, 0, 0, 2, 3) == 119

...although with the class below, the thought process was too messy to try to tell as a story. I knocked out this class while watching both *Wicked* movies for the first time. There were a number of false starts and dead ends.

For the depth-first search I decided to choose the next rectangle based on an empty square rather than an unused number, because it saved me the step of explicitly tracking which numbers have been used. (Although I did leave `Rs` in there for future optimization tests.)

Currently there's no validation that the input is rectangular.

In [5]:
from collections import defaultdict
from math import log2

class Grid:
    def __init__(self, s):
        self.s = s
        lines = s.strip().split('\n')
        self.H = len(lines)
        self.W = len(lines[0])
        self.MAX = 2 ** (self.H * self.W) - 1

        self.Ns = {}  # Number for a given Square
        for r, row in enumerate(lines):
            for c, char in enumerate(row.split(',')):
                if char:
                    self.Ns[2 ** (r * self.W + c)] = int(char)  # convert row and column to bit

        self.Rs = defaultdict(list)  # list of possible Rectangles containing a given Number
        self.Qs = defaultdict(list)  # list of possible Rectangles containing a given Square
        for this_p, n in self.Ns.items():
            r, c = divmod(int(log2(this_p)), self.W)  # convert bit back to row and column
            other_ps = sum(self.Ns) - this_p
            F = factors(n)
            for h, w in zip(F, reversed(F)):  # all possible Rectangle shapes
                for dh in range(h):           # all possible horizontal offsets
                    for dw in range(w):       # all possible vertical offsets
                        rectangle = self.bitmask_rectangle(r - dh, c - dw, h, w)
                        if rectangle and not rectangle & other_ps:  # has set bits and doesn't overlap any other Number
                            self.Rs[this_p].append(rectangle)
                            for r0 in range(r - dh, r - dh + h):
                                for c0 in range(c - dw, c - dw + w):
                                    self.Qs[2 ** (r0 * self.W + c0)].append(rectangle)

    def bitmask_rectangle(self, r, c, h, w):
        if r < 0 or c < 0 or r + h > self.H or c + w > self.W:
            return 0
        result = 0
        for r0 in range(r, r + h):
            for c0 in range(c, c + w):
                result += 2 ** (r0 * self.W + c0)
        return result

    def dfs(self, rs: set):
        current = sum(rs)  # union of accumulated rectangles
        if current == self.MAX:
            yield rs
        # continue from some empty square; for now, using lowest zero bit
        lowest_zero_bit = ~current & (current + 1)
        for rectangle in self.Qs[lowest_zero_bit]:
            if not rectangle & current:
                next_rs = rs | {rectangle}
                yield from self.dfs(next_rs)

    def solve(self):
        yield from self.dfs(set())

    def __repr__(self):
        output = []
        spacer_row = '+' + '--+' * self.W
        output.append(spacer_row)
        for line in self.s.strip().split('\n'):
            row = [f"{value:>2}" for value in line.split(',')]
            output.append('|' + '|'.join(row) + '|')
            output.append(spacer_row)
        return '\n'.join(output)

Testing it on the Wikipedia puzzle:

In [6]:
test_grid = Grid(sample_grid_cleaned)

In [7]:
test_solutions = [solution for solution in test_grid.solve()]

In [8]:
print(test_grid)
print(f"Number of solutions: {len(test_solutions)}")

+--+--+--+--+--+--+--+--+--+--+--+
|  | 4|  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2| 4|  |  | 3|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |16|  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 6| 5| 2| 4|  |  | 4|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  | 2| 5|  |  | 2|  | 7|
+--+--+--+--+--+--+--+--+--+--+--+
|  |  | 2|  |  | 2| 4|  | 2|  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |16|  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |12|  | 6|  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
| 2|  |  |  |  |  |  |  |  | 9|  |
+--+--+--+--+--+--+--+--+--+--+--+
|  |  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+
Number of solutions: 1


Well, I didn't necessarily expect there would be just a single solution. I hadn't tried to solve it by hand :)

A few reality checks. The possible rectangles for the 2 in the second row:

In [9]:
for n in test_grid.Rs[2 ** 11]:
    print(f"{bin(n)[2:]:>30}")

       10000000000100000000000
                  100000000001


The possible rectangles for the square in the upper-left corner:

In [10]:
for n in test_grid.Qs[2 ** 0]:
    print(f"{bin(n)[2:]:>30}")

                          1111
                  100000000001


The bitmasks for the rectangles in the solution:

In [11]:
test_solutions

[{15,
  4196352,
  25178112,
  67141648,
  550024380480,
  16500731414400,
  70403103916032,
  140806241402880,
  281612415664128,
  563224965611552,
  13510798882111488,
  432345564227567616,
  1729382256910270464,
  6920906727361609728,
  27670116110564327424,
  302379100949042559975424,
  154213304716685954745630720,
  309636199371819590004768768,
  1298708039933821021833372433907712,
  38961250482564925295353234760663040,
  124676001544207760945130351234121728,
  1163642681079272435487883278185136128,
  1329877349959700883100624741111693312}]

Before anything else, I tried a simple case where I knew there were multiple solutions:

In [12]:
s2 = """
2,
,2
"""

In [13]:
test_grid_2x2 = Grid(s2)
test_solutions_2x2 = [solution for solution in test_grid_2x2.solve()]

In [14]:
print(test_grid_2x2)
print(f"Number of solutions: {len(test_solutions_2x2)}")

+--+--+
| 2|  |
+--+--+
|  | 2|
+--+--+
Number of solutions: 2


In [15]:
test_solutions_2x2

[{3, 12}, {5, 10}]

I'd like to test more examples of decent size, particularly some where I know there are multiple solutions. But first, can I visualize the collection of rectangles?

In [16]:
import random
from string import ascii_letters, digits
from IPython.display import display, HTML

def visualize_ascii(grid, solution, seed=None):
    if seed is not None:
        random.seed(seed)
    chars = list(digits + ascii_letters)
    random.shuffle(chars)

    labels = [None for _ in range(grid.H * grid.W)]
    for i, r in enumerate(solution):
        bitstring = bin(r)[2:][::-1]
        for j, bit in enumerate(bitstring):
            if bit == '1':
                labels[j] = i
    
    i = 0
    n = 1
    lines = []
    for r in range(grid.H):
        line = []
        for c in range(grid.W):
            if n in grid.Ns:
                line.append(f"<b>{chars[labels[i]]}</b>")
            else:
                line.append(chars[labels[i]])
            i += 1
            n *= 2
        lines.append(''.join(line))
    full_output = '\n'.join(lines)
    html_string = f"<pre style='font-family: monospace; line-height: 1.1;'>{full_output}</pre>"
    display(HTML(html_string))

In [17]:
visualize_ascii(test_grid, test_solutions[0], seed='test_grid')

In [18]:
for i, solution in enumerate(test_solutions_2x2, start=1):
    visualize_ascii(test_grid_2x2, solution, seed='2x2')